# Trabajo Practico 2
## Ejercicios 1 y 4

In [1]:
import numpy as np
import simpy

### Funciones y constantes comunes

In [2]:
def calcular_promedio(numeros):
    return sum(numeros) / len(numeros)

In [24]:
def mostrar_resultado(resultado):
    tiempo_medio_espera, fraccion_sin_espera, tasa_finalizacion = resultado
    print("Tiempo medio de espera: {} segundos".format(tiempo_medio_espera))
    print("Fraccion de solicitudes sin esperar: {}".format(fraccion_sin_espera))
    print("Tasa de finalizacion: {} solicitudes finalizadas por segundo".format(tasa_finalizacion))

In [23]:
MUESTRA = 100000
N_SIMULACIONES = 100
PROBA_BASE_A = 0.6
MEDIA_BASE_A = 0.7
MEDIA_BASE_B = 0.95
MEDIA_BASE = 0.8

## Ejercicio 1

Simule el siguiente problema.
Se está diseñando un web service, el cual cada vez que es invocado consulta a una base de datos.
El tiempo que transcurre entre cada llamada al servicio se puede modelar según una distribución exponencial con media 𝜇
Considerar 𝜇 = 1, 2 𝑦 4 𝑠𝑒𝑔𝑢𝑛𝑑𝑜𝑠

Realizar 100 simulaciones de cada modelo, con 100000 solicitudes procesadas, y determinar:
- El tiempo medio de espera entre que la solicitud llega y puede ser procesada (suponer que ninguna conexión se
cae por timeout).
- La fracción de las solicitudes que no esperaron para ser procesadas.
- La tasa de finalización de consultas (consultas finalizadas por segundo)
- ¿Qué solución le recomienda? Justifique

In [12]:
def tiempos_de_arribo(media, cantidad_ensayos):
    z = np.random.exponential(media, cantidad_ensayos)
    return np.concatenate(([0], np.cumsum(z)), axis=None)

#### Alternativa 1
Se utilizan 2 bases de datos distribuidas.
Con probabilidad 𝑝 = 0.6 las solicitudes son atendidas por la base A y con probabilidad 𝑞 = 1 − 𝑝 son atendidos por la
base de datos B.
El tiempo que demora cada base de datos en atender una solicitud sigue una distribución exponencial con medias,
𝜇1 = 0,7 𝑠𝑒𝑔 y 𝜇2 = 0,95 𝑠𝑒𝑔 respectivamente.

In [15]:
def tiempos_en_base(arribos, media):
    t_actual = 0
    tiempos_espera = []
    tiempos_procesado = []
    for t_arribo in arribos:
        t_base = np.random.exponential(media)
        if t_arribo < t_actual:
            t_espera = t_actual - t_arribo
            tiempos_espera.append(t_espera)
            t_procesado = t_arribo + t_espera + t_base
            tiempos_procesado.append(t_procesado)
            t_actual += t_base
        else:
            t_espera = 0
            tiempos_espera.append(t_espera)
            t_procesado = t_arribo + t_base
            tiempos_procesado.append(t_procesado)
            t_actual = t_arribo + t_base
    return tiempos_espera, tiempos_procesado

In [16]:
def alternativa_1(media_arribos, solicitudes_procesadas):
    tiempos_arribo = tiempos_de_arribo(media_arribos, solicitudes_procesadas)
    tiempos_a = []
    tiempos_b = []
    for t in tiempos_arribo:
        bdd = np.random.uniform()
        t_p = t
        t_e = 0
        if bdd < PROBA_BASE_A:
            tiempos_a.append(t)
        else:
            tiempos_b.append(t)
    tiempos_espera_a, tiempos_procesado_a = tiempos_en_base(tiempos_a, MEDIA_BASE_A)
    tiempos_espera_b, tiempos_procesado_b = tiempos_en_base(tiempos_b, MEDIA_BASE_B)
    return tiempos_a + tiempos_b, tiempos_procesado_a + tiempos_procesado_b, tiempos_espera_a + tiempos_espera_b

In [17]:
def simular(media_arribos, cantidad_simulaciones, muestra, alternativa):
    tiempos_medios_espera_resultado = []
    fraccion_sin_espera_resultado = []
    tasa_finalizacion_resultado = []
    for i in range(cantidad_simulaciones):
        tiempos_arribo, tiempos_procesado, tiempos_espera = alternativa(media_arribos, muestra)
        tiempos_medios_espera_resultado.append(sum(tiempos_espera)/muestra)
        sin_espera = list(filter(lambda x: x==0, tiempos_espera))
        fraccion_sin_espera = len(sin_espera) / muestra
        fraccion_sin_espera_resultado.append(fraccion_sin_espera)
        tasa_finalizacion = muestra / max(tiempos_procesado)
        tasa_finalizacion_resultado.append(tasa_finalizacion)
    tiempo_medio_total = calcular_promedio(tiempos_medios_espera_resultado)
    fraccion_sin_espera_total = calcular_promedio(fraccion_sin_espera_resultado)
    tasa_finalizacion_total = calcular_promedio(tasa_finalizacion_resultado)
    return tiempo_medio_total, fraccion_sin_espera_total, tasa_finalizacion_total

In [20]:
MEDIA_ARRIBOS = 1

mostrar_resultado(simular(MEDIA_ARRIBOS, N_SIMULACIONES, MUESTRA, alternativa_1))

Tiempo medio de espera: 0.5390863753426424 segundos
Fraccion de solicitudes sin esperar: 0.5954540999999999
Tasa de finalizacion: 1.0000859109275324 solicitudes finalizadas por segundo


In [21]:
MEDIA_ARRIBOS = 2

mostrar_resultado(simular(MEDIA_ARRIBOS, N_SIMULACIONES, MUESTRA, alternativa_1))

Tiempo medio de espera: 0.20036124773930744 segundos
Fraccion de solicitudes sin esperar: 0.7980895999999998
Tasa de finalizacion: 0.5000551180016598 solicitudes finalizadas por segundo


In [22]:
MEDIA_ARRIBOS = 4

mostrar_resultado(simular(MEDIA_ARRIBOS, N_SIMULACIONES, MUESTRA, alternativa_1))

Tiempo medio de espera: 0.08930493735516226 segundos
Fraccion de solicitudes sin esperar: 0.8990277
Tasa de finalizacion: 0.24998896240985002 solicitudes finalizadas por segundo


#### Alternativa 2
Utilizar 1 base de datos central.
En este caso la demora en resolver una solicitud sigue una distribución exponencial con 𝜇 = 0,8 𝑠𝑒𝑔𝑢𝑛𝑑𝑜

In [25]:
def alternativa_2(media_arribos, solicitudes_procesadas):
    tiempos_arribo = tiempos_de_arribo(media_arribos, solicitudes_procesadas)
    tiempos_espera,tiempos_procesado = tiempos_en_base(tiempos_arribo, 0.8)
    return tiempos_arribo, tiempos_procesado, tiempos_espera

In [26]:
MEDIA_ARRIBOS = 1

mostrar_resultado(simular(MEDIA_ARRIBOS, N_SIMULACIONES, MUESTRA, alternativa_2))

Tiempo medio de espera: 3.1936877250632194 segundos
Fraccion de solicitudes sin esperar: 0.20024400000000006
Tasa de finalizacion: 0.9994393049753896 solicitudes finalizadas por segundo


In [27]:
MEDIA_ARRIBOS = 2

mostrar_resultado(simular(MEDIA_ARRIBOS, N_SIMULACIONES, MUESTRA, alternativa_2))

Tiempo medio de espera: 0.5327471406618114 segundos
Fraccion de solicitudes sin esperar: 0.6000401
Tasa de finalizacion: 0.5000104982012297 solicitudes finalizadas por segundo


In [28]:
MEDIA_ARRIBOS = 4

mostrar_resultado(simular(MEDIA_ARRIBOS, N_SIMULACIONES, MUESTRA, alternativa_2))

Tiempo medio de espera: 0.20026830737255819 segundos
Fraccion de solicitudes sin esperar: 0.7998767999999998
Tasa de finalizacion: 0.2499652652882094 solicitudes finalizadas por segundo


#### Conclusiones

Para concluir cuál alternativa es la que se recomienda se deben observar los distintos valores obtenidos:
- Tiempo medio de espera: la alternativa 1 tiene menor tiempo medio de espera para todas las medias propuestas
- Fracción de solicitudes sin esperar: la alternativa 1 tiene mayor fracción de solicitudes sin esperar para todas las medias propuestas
- Tasa de finalización: son similares en ambas alternativas

En conclusión, se recomienda la alternativa 1 ya que se obtuvieron mejores resultados en los distintos valores analizados. 

## Ejercicio 4

Implementar las 2 alternativas del Ejercicio 1 utilizando SimPy.

Comparar los resultados obtenidos con el ejercicio 1.

In [29]:
def request(env, counter, media_base, tiempos_espera, tiempos_procesado):
    tiempo_arribo = env.now

    with counter.request() as req:
        yield req
        tiempo_actual = env.now
        tiempo_espera = tiempo_actual - tiempo_arribo
        t_base = np.random.exponential(media_base)
        yield env.timeout(t_base)
        tiempos_espera.append(tiempo_espera)
        tiempos_procesado.append(tiempo_actual + tiempo_espera + t_base)

#### Alternativa 1
Se utilizan 2 bases de datos distribuidas.
Con probabilidad 𝑝 = 0.6 las solicitudes son atendidas por la base A y con probabilidad 𝑞 = 1 − 𝑝 son atendidos por la
base de datos B.
El tiempo que demora cada base de datos en atender una solicitud sigue una distribución exponencial con medias,
𝜇1 = 0,7 𝑠𝑒𝑔 y 𝜇2 = 0,95 𝑠𝑒𝑔 respectivamente.

In [31]:
def source_1(env, muestra, media_arribos, proba_a, counter_a, counter_b, media_base_a, media_base_b, tiempos_espera, tiempos_procesado):
    for i in range(muestra):
        bdd = np.random.uniform()
        if bdd < proba_a:
            r = request(env, counter_a, media_base_a, tiempos_espera, tiempos_procesado)
        else:
            r = request(env, counter_b, media_base_b, tiempos_espera, tiempos_procesado)
        env.process(r)
        t = np.random.exponential(media_arribos)
        yield env.timeout(t)
        
def alternativa_1(muestra, media_arribos, proba_a, media_base_a, media_base_b, n_simulaciones):
    tiempos_espera_medios_resultado = []
    fraccion_sin_espera_resultado = []
    taza_finalizacion_resultado = []
    
    for _ in range(n_simulaciones):
        tiempos_espera = []
        tiempos_procesado = []
        env = simpy.Environment()
        counter_a = simpy.Resource(env, capacity=1)
        counter_b = simpy.Resource(env, capacity=1)
        env.process(source_1(env, muestra, media_arribos, proba_a, counter_a, counter_b, media_base_a, media_base_b, tiempos_espera, tiempos_procesado))
        env.run()        
        
        sin_espera = list(filter(lambda x: x==0, tiempos_espera))
        fraccion_sin_espera = len(sin_espera) / muestra
        fraccion_sin_espera_resultado.append(fraccion_sin_espera)
        
        taza_finalizacion = muestra / max(tiempos_procesado)
        taza_finalizacion_resultado.append(taza_finalizacion)
        
        tiempos_espera_medios_resultado.append(calcular_promedio(tiempos_espera))
    return tiempos_espera_medios_resultado, fraccion_sin_espera_resultado, taza_finalizacion_resultado

In [40]:
def simular_alternativa_1(muestra, media_arribos, proba_base_a, media_base_a, media_base_b, n_simulaciones):
    tiempos_espera_medios, fraccion_sin_espera_resultado, taza_finalizacion_resultado = alternativa_1(muestra, media_arribos, proba_base_a, media_base_a, media_base_b, n_simulaciones)
    tiempo_medio_espera = calcular_promedio(tiempos_espera_medios)
    fraccion_sin_espera = calcular_promedio(fraccion_sin_espera_resultado)
    taza_finalizacion = calcular_promedio(taza_finalizacion_resultado)
    return tiempo_medio_espera, fraccion_sin_espera, taza_finalizacion

In [36]:
MEDIA_ARRIBOS = 1

mostrar_resultado(simular_alternativa_1(MUESTRA, MEDIA_ARRIBOS, PROBA_BASE_A, MEDIA_BASE_A, MEDIA_BASE_B, N_SIMULACIONES))

Tiempo medio de espera: 0.5372692268373614 segundos
Fraccion de solicitudes sin esperar: 0.5958693
Tasa de finalizacion: 1.0002647641165139 solicitudes finalizadas por segundo


In [37]:
MEDIA_ARRIBOS = 2

mostrar_resultado(simular_alternativa_1(MUESTRA, MEDIA_ARRIBOS, PROBA_BASE_A, MEDIA_BASE_A, MEDIA_BASE_B, N_SIMULACIONES))

Tiempo medio de espera: 0.20065847230400766 segundos
Fraccion de solicitudes sin esperar: 0.7979780000000001
Tasa de finalizacion: 0.5001091709594859 solicitudes finalizadas por segundo


In [38]:
MEDIA_ARRIBOS = 4

mostrar_resultado(simular_alternativa_1(MUESTRA, MEDIA_ARRIBOS, PROBA_BASE_A, MEDIA_BASE_A, MEDIA_BASE_B, N_SIMULACIONES))

Tiempo medio de espera: 0.0891406066063754 segundos
Fraccion de solicitudes sin esperar: 0.8989565999999999
Tasa de finalizacion: 0.25004003450449724 solicitudes finalizadas por segundo


#### Alternativa 2

Utilizar 1 base de datos central. En este caso la demora en resolver una solicitud sigue una distribución exponencial con 𝜇 = 0,8 𝑠𝑒𝑔𝑢𝑛𝑑𝑜

In [39]:
def source_2(env, muestra, media_arribos, counter, media_base, tiempos_espera, tiempos_procesado):
    for i in range(muestra):
        r = request(env, counter, media_base, tiempos_espera, tiempos_procesado)
        env.process(r)
        t = np.random.exponential(media_arribos)
        yield env.timeout(t)
        
def alternativa_2(muestra, media_arribos, media_base, n_simulaciones):
    tiempos_espera_medios_resultado = []
    fraccion_sin_espera_resultado = []
    taza_finalizacion_resultado = []
    
    for _ in range(n_simulaciones):
        tiempos_espera = []
        tiempos_procesado = []
        env = simpy.Environment()
        counter = simpy.Resource(env, capacity=1)
        env.process(source_2(env, muestra, media_arribos, counter, media_base, tiempos_espera, tiempos_procesado))
        env.run()        
        
        sin_espera = list(filter(lambda x: x==0, tiempos_espera))
        fraccion_sin_espera = len(sin_espera) / muestra
        fraccion_sin_espera_resultado.append(fraccion_sin_espera)
        
        taza_finalizacion = muestra / max(tiempos_procesado)
        taza_finalizacion_resultado.append(taza_finalizacion)
        
        tiempos_espera_medios_resultado.append(calcular_promedio(tiempos_espera))
    return tiempos_espera_medios_resultado, fraccion_sin_espera_resultado, taza_finalizacion_resultado

In [41]:
def simular_alternativa_2(muestra, media_arribos, media_base, n_simulaciones):
    tiempos_espera_medios, fraccion_sin_espera_resultado, taza_finalizacion_resultado = alternativa_2(muestra, media_arribos, media_base, n_simulaciones)
    tiempo_medio_espera = calcular_promedio(tiempos_espera_medios)
    fraccion_sin_espera = calcular_promedio(fraccion_sin_espera_resultado)
    taza_finalizacion = calcular_promedio(taza_finalizacion_resultado)
    return tiempo_medio_espera, fraccion_sin_espera, taza_finalizacion

In [42]:
MEDIA_ARRIBOS = 1

mostrar_resultado(simular_alternativa_2(MUESTRA, MEDIA_ARRIBOS, MEDIA_BASE, N_SIMULACIONES))

Tiempo medio de espera: 3.197403843277221 segundos
Fraccion de solicitudes sin esperar: 0.20017799999999997
Tasa de finalizacion: 0.9997496752377522 solicitudes finalizadas por segundo


In [43]:
MEDIA_ARRIBOS = 2

mostrar_resultado(simular_alternativa_2(MUESTRA, MEDIA_ARRIBOS, MEDIA_BASE, N_SIMULACIONES))

Tiempo medio de espera: 0.5344411593051502 segundos
Fraccion de solicitudes sin esperar: 0.5996451000000002
Tasa de finalizacion: 0.5002928668003748 solicitudes finalizadas por segundo


In [44]:
MEDIA_ARRIBOS = 4

mostrar_resultado(simular_alternativa_2(MUESTRA, MEDIA_ARRIBOS, MEDIA_BASE, N_SIMULACIONES))

Tiempo medio de espera: 0.2001677007494556 segundos
Fraccion de solicitudes sin esperar: 0.7999754
Tasa de finalizacion: 0.25004058959819725 solicitudes finalizadas por segundo


#### Conclusiones

Se puede observar que los resultados para ambas alternativas son muy similares a los encontrados sin el uso de simpy. Por lo tanto, al igual que en el ejercicio 1 la alternativa menos costosa es la 1 por las mismas razones.